# Quaternion Tests

The SPT-3G framework uses quaternions to encode pointing directions (e.g. boresight RA/DEC) as well as detector focal plane offsets. However, the methods used to map between RA/DEC <--> quaternions or lat/lon <--> quaterions are not always clear.

SPT-3G's ``spt3g.core.maps`` package contains some methods relating to map pointing ([see documentation](https://cmb-s4.github.io/spt3g_software/moddoc_maps.html#map-pointing)):

    "This package also provides functions and pipline modules for creating and manipulating the quaternions necessary for mapmaking. In general, there are two forms of quaternions that are used throughout the code: pointing quaternions and rotation quaternions."



The Simons Observatory's ``so3g.proj`` module ([see documentation](https://so3g.readthedocs.io/en/latest/proj.html)) contains methods which map between quaternions and multiple coordinate systems. Namely:
- ``so3g.proj.quat.rotation_iso(theta, phi, psi=None)``
- ``so3g.proj.quat.rotation_lonlat(lon, lat, psi=0.0)``
- ``so3g.proj.quat.rotation_xieta(xi, eta, gamma=0)``

The inverse methods also exist:
- ``so3g.proj.quat.decompose_iso(q)``
- ``so3g.proj.quat.decompose_lonlat(q)``
- ``so3g.proj.quat.decompose_xieta(q)``

In [2]:
from spt3g import core, maps
from spt3g.core import G3Units as U
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from so3g import proj

## 1. SPT-3G Methods

As described in the docs linked above, quaternions can represent either:

1. A pointing quaternion. In this case, the information is encoded in the vector component of the quaternion, i.e., the second, third and fourth components. I believe a pointing quaternion would typically have magnitude $||q||=1$ and zero real component. However, we will show below that the method ``maps.quat_to_ang`` does not enforce this.

2. A rotation quaternion. A rotation quaternion must be a unit quaternion (i.e., $||q||=1$ ). According to the ``spt3g.maps`` [docs](https://cmb-s4.github.io/spt3g_software/moddoc_maps.html#rotation-quaternions), A rotation quaternion $q_r$ be applied to a pointing quaternion $q_p$ by the formula:  $$q_{p,rot} = q_r * q_p / q_r\,.$$ A rotation quaternion can be interpreted as a rotation by an angle $\theta$ about a direction $\hat{\textbf{u}}$ where $\hat{\textbf{u}}=u_x\hat{\textbf{i}} + u_y\hat{\textbf{j}} + u_z\hat{\textbf{k}}$. In this case, the quaternion will be given by: $$q=\cos(\frac{\theta}{2}) + \sin(\frac{\theta}{2}) \hat{u} = \cos(\frac{\theta}{2}) + \sin(\frac{\theta}{2}) (u_x\hat{\textbf{i}} + u_y\hat{\textbf{j}} + u_z\hat{\textbf{k}})\,.$$

### 1.1 ``ang_to_quat``

How do angles map to (pointing quaternions)?

In [18]:
# Test 1 - How does spt3g.maps map between RA (alpha) / DEC (delta) and quaternions?
maps.ang_to_quat(0, 0)

spt3g.core.quat(0,1,0,0)

We see from the above test that for spt3g pointing quaternions, RA/DEC = 0deg/0deg is mapped to the vector (1, 0, 0). This means that the positive x direction (AKA,  $\hat{\textbf{i}}$) points in the direction where $\alpha=0$ and $\delta=0$.

In the next cells, we can confirm that $\hat{\textbf{j}}$ and $\hat{\textbf{k}}$ correspond to the directions shown in the sketch below.

![test](ra_dec_sketch.jpg)

In [16]:
# Test 2 - Confirming the y (j_hat) and direction for spt3g pointing quaternions
y_pointing = maps.ang_to_quat(np.pi/2, 0)
assert np.allclose([y_pointing.b, y_pointing.c, y_pointing.d], np.array([0, 1, 0]))
print(y_pointing)

(0,6.12323e-17,1,0)


In [15]:
# Test 3 - Confirming the z (k_hat) and direction for spt3g pointing quaternions
z_pointing = maps.ang_to_quat(0, np.pi/2)
assert np.allclose([z_pointing.b, z_pointing.c, z_pointing.d], np.array([0, 0, 1]))
print(z_pointing)

(0,6.12323e-17,0,1)


### 1.2 ``quat_to_ang``

How do pointing quaternions map to RA/DEC angles?

In [20]:
# Test 4 - inverting the known x, y and z pointing quaternions from section 1.1
ra, dec = maps.quat_to_ang(core.quat(0, 1, 0, 0))
print(f"X direction -- RA (deg): {ra * 180 / np.pi}, DEC (deg): {dec * 180 / np.pi}")

ra, dec = maps.quat_to_ang(core.quat(0, 0, 1, 0))
print(f"Y direction -- RA (deg): {ra * 180 / np.pi}, DEC (deg): {dec * 180 / np.pi}")

ra, dec = maps.quat_to_ang(core.quat(0, 0, 0, 1))
print(f"Z direction -- RA (deg): {ra * 180 / np.pi}, DEC (deg): {dec * 180 / np.pi}")

X direction -- RA (deg): 0.0, DEC (deg): 0.0
Y direction -- RA (deg): 90.0, DEC (deg): 0.0
Z direction -- RA (deg): 0.0, DEC (deg): 90.0


In [44]:
# Test 5 - pointing quaternions with non-zero real components
for realval in (0., 1., 100., 1000., -10., 0.333, np.pi):
    ra, dec = maps.quat_to_ang(core.quat(realval, 1, 0, 0))
    print(f"Real component: {realval:>6.5}, RA (deg): {ra * 180 / np.pi}, DEC (deg): {dec * 180 / np.pi}")
# conclusion: quat_to_ang ignores the real component of quaternions,
# as they are expected to be pointing quaternions where the information is only in the vector component

Real component:    0.0, RA (deg): 0.0, DEC (deg): 0.0
Real component:    1.0, RA (deg): 0.0, DEC (deg): 0.0
Real component:  100.0, RA (deg): 0.0, DEC (deg): 0.0
Real component: 1000.0, RA (deg): 0.0, DEC (deg): 0.0
Real component:  -10.0, RA (deg): 0.0, DEC (deg): 0.0
Real component:  0.333, RA (deg): 0.0, DEC (deg): 0.0
Real component: 3.1416, RA (deg): 0.0, DEC (deg): 0.0


In [49]:
# Test 6 - pointing quaternions with non-unit norms
def norm(q): return (q.a**2 + q.b**2 + q.c**2 + q.d**2)**0.5
test_quats = [
    (0, np.sqrt(2)/2, np.sqrt(2)/2, 0),  # unit norm
    (0, 1, 1, 0),                        # non-unit norm
    (0, 1000, 1000, 0),                  # non-unit norm
    (0, -np.sqrt(2)/2, -np.sqrt(2)/2, 0) # unit norm, opposite direction
]
for quat_args in test_quats:
    q = core.quat(*quat_args)
    ra, dec = maps.quat_to_ang(q)
    print(f"{q=} -- RA (deg): {ra * 180 / np.pi}, DEC (deg): {dec * 180 / np.pi}")
# conclusion: quat_to_ang treats pointing quaternions as if their vector component were normalized to 1

q=spt3g.core.quat(0,0.707107,0.707107,0) -- RA (deg): 45.0, DEC (deg): 0.0
q=spt3g.core.quat(0,1,1,0) -- RA (deg): 45.0, DEC (deg): 0.0
q=spt3g.core.quat(0,1000,1000,0) -- RA (deg): 45.0, DEC (deg): 0.0
q=spt3g.core.quat(0,-0.707107,-0.707107,0) -- RA (deg): -135.0, DEC (deg): 0.0


## Appendix: Misc Experiments / Testing

In [6]:
def norm(q):
    return (q.a**2 + q.b**2 + q.c**2 + q.d**2)**0.5

q_to_gal = maps.get_fk5_j2000_to_gal_quat()



In [10]:
maps.get_origin_rotator(0, np.pi / 4)

spt3g.core.quat(0.92388,0,-0.382683,0)

In [27]:
angle = maps.get_rot_ang(core.quat(0, 1, 0, 0), core.quat(0.92388,0,-0.382683,0))
angle * 180 / np.pi

1.2074182697257333e-06

    spt3g.maps.offsets_to_quat

    offsets_to_quat( (float)x, (float)y) -> Quat :

    Returns the vector quaternion (0,1,0,0) rotated by the given x and y offsets. Equivalent to t * quat(0,1,0,0) / t, where t = get_origin_rotator(x, -y)

Why negative y?


In [47]:
maps.quathelpers.quat_to_ang(core.quat(0, 1, 1, 0))

(np.float64(0.7853981633974483), np.float64(0.0))

In [50]:
maps.quathelpers.ang_to_quat(3/2*np.pi, -np.pi/4)

spt3g.core.quat(0,-1.29893e-16,-0.707107,-0.707107)

In [10]:
as_0, ds_0 = (0, 0)        # positive x
as_1, ds_1 = (0, np.pi/2)  # positive z

ae_0, de_0 = (0, np.pi/2)  # positive z
ae_1, de_1 = (np.pi, 0)    # negative x

# the transform_quat should be a -90 degree rotation about the y axis, or a 90 degree rotation about the -y axis
# either way, we expect the quaternion to be equal to cos(90deg/2) + sin(-90deg/2)<0, 1, 0>
ninety_degrees = np.pi/2
q_expected = core.quat(np.cos(ninety_degrees/2), 0, -np.sin(ninety_degrees/2), 0)

q_obtained = maps.get_transform_quat(as_0, ds_0, ae_0, de_0, as_1, ds_1, ae_1, de_1)

print(f"{q_expected=}, {q_obtained=}")

q_expected=spt3g.core.quat(0.707107,0,-0.707107,0), q_obtained=spt3g.core.quat(0.707107,0,-0.707107,0)


In [11]:
as_0, ds_0 = (0, 0)        # positive x
as_1, ds_1 = (0, np.pi/2)  # positive z

ae_0, de_0 = (0, np.pi/2)  # positive z
ae_1, de_1 = (0, 0)        # positive x

# the transform_quat should be a +-180 degree rotation about the axis in between positive x and positive z
# that axis would be given by RA (alpha) = 0 and DEC (delta) = pi/4, so we use ang_to_quat to get the expected axis
rotation_axis = maps.ang_to_quat(0, np.pi/4)
one_eighty_degrees = np.pi
q_expected = core.quat(
    np.cos(one_eighty_degrees/2),
    np.sin(one_eighty_degrees/2) * rotation_axis.b,
    np.sin(one_eighty_degrees/2) * rotation_axis.c,
    np.sin(one_eighty_degrees/2) * rotation_axis.d
)

q_obtained = maps.get_transform_quat(as_0, ds_0, ae_0, de_0, as_1, ds_1, ae_1, de_1)
print(f"{rotation_axis=}")
print(f"{q_expected=}, {q_obtained=}")

rotation_axis=spt3g.core.quat(0,0.707107,0,0.707107)
q_expected=spt3g.core.quat(6.12323e-17,0.707107,0,0.707107), q_obtained=spt3g.core.quat(4.32978e-17,0.707107,-4.32978e-17,0.707107)


In [21]:
def assert_quats_equal(q1, q2, msg="q1 and q2 are not equal!"):
    assert np.allclose([q1.a, q1.b, q1.c, q1.d], [q2.a, q2.b, q2.c, q2.d]), msg


# this time we test an impossible configuration -- if the angle between s_0 and s_1 is different from the angle between e_0 and e_1,
# there should be no valid rotation solution which "will take (as_0,ds_0) to (ae_0, de_0) and (as_1, ds_1) to (ae_1, de_1)"

as_0, ds_0 = (0, 0)        # positive x
as_1, ds_1 = (0, np.pi/2)  # positive z

ae_0, de_0 = (0, np.pi/2)  # positive z
ae_1, de_1 = (0, -np.pi/2) # negative z

# I have no idea what to expect, except an error
q_obtained = maps.get_transform_quat(as_0, ds_0, ae_0, de_0, as_1, ds_1, ae_1, de_1)
print(f"{q_obtained=}")

# the result: (0, sqrt(2)/2, 0, sqrt(2)/2)
# this corresponds to a rotation of 180 degrees (since the real component is zero, theta/2 is likely pi/2) about the axis between +x and +z.
# In fact, this is the same output as in the last test. The result does map as_0, ds_0 to ae_0, de_0 but does not do the same for as_1, ds_1 to ae_1, de_1


assert_quats_equal(
    q_obtained * maps.ang_to_quat(as_0, ds_0) / q_obtained,
    maps.ang_to_quat(ae_0, de_0)
)

assert_quats_equal(
    q_obtained * maps.ang_to_quat(as_1, ds_1) / q_obtained,
    maps.ang_to_quat(ae_1, de_1),
    "applying q_obtained to as_1/ds_1 does not map to ae_1/de_1!"
)

q_obtained=spt3g.core.quat(4.32978e-17,0.707107,-4.32978e-17,0.707107)


AssertionError: applying q_obtained to as_1/ds_1 does not map to ae_1/de_1!

In [24]:
# again, it is no surprise that this doesn't work -- the angles between the vectors are not the same.
# recalling u dot v = |u||v|cos(theta)
def norm(q):
    return (q.a**2 + q.b**2 + q.c**2 + q.d**2)**0.5
def angle(q1, q2):
    """Determine the angle in radians between two pointing quaternions."""
    assert q1.a == 0, "pointing quaternion must only have vector (non-real) component!"
    assert q2.a == 0, "pointing quaternion must only have vector (non-real) component!"
    return np.acos(np.dot([q1.b, q1.c, q1.d], [q2.b, q2.c, q2.d]) / norm(q1) / norm(q2))

print(f'{angle(maps.ang_to_quat(as_0, ds_0), maps.ang_to_quat(as_1, ds_1))=}')
print(f'{angle(maps.ang_to_quat(ae_0, de_0), maps.ang_to_quat(ae_1, de_1))=}')

angle(maps.ang_to_quat(as_0, ds_0), maps.ang_to_quat(as_1, ds_1))=np.float64(1.5707963267948966)
angle(maps.ang_to_quat(ae_0, de_0), maps.ang_to_quat(ae_1, de_1))=np.float64(3.141592653589793)
